## Imports and Data Load

In [1]:
from utils_v2 import *
from UNet import *

In [2]:
import gc

gc.collect()

0

## Dataset

In [3]:
ds = xr.open_dataset('data_850/2022_850_SA.nc')
ds.load()

weather_data = WeatherData(ds, window_size=3)
weather_data.subset_data()
weather_data.window_dataset()

c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


## Model

In [4]:
def build_unet(input_shape, output_channels = 1):
    input_data = Input(input_shape)

    s1, p1 = encoder_block(input_data, 16)
    s2, p2 = encoder_block(p1, 32)
    # s3, p3 = encoder_block(p2, 256)
    # s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p2, 64) #Bridge

    # d1 = decoder_block(b1, s4, 512)
    # d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(b1, s2, 32)
    d4 = decoder_block(d3, s1, 16)

    time_input = Input(shape=(2,))

    outputs = Conv2D(output_channels, (1, 1), activation='sigmoid')(d4)
    
    model = Model(inputs=[input_data, time_input], outputs=outputs, name="U-Net-Forcings")
    return model


## Preprocessing

In [5]:
model_class = WeatherMLModel()

features, targets, forcings, time_values = weather_data.return_data()
model_class.load_data(features=features, targets=targets, forcings=forcings, time_values=time_values)
model_class.split_data()

model = build_unet(model_class.X_train.shape[1:], model_class.y_train.shape[-1])
model_class.assign_model(model)

Class setup done...
Data loaded...
Splitting...
Shuffling...


Model assigned...


## Plotting predictions

In [6]:
model_class.load_model('models/09_11_10_16.h5')

In [7]:
model_class.plot_predictions(0, levels=10, frame_rate=10)

1/1 [==============================] - 1s 1s/step
